# Unindo os dados das avaliações

*todos os prints do dataframe que continham valores expostos foram deletados para preservar os dados.

In [366]:
import pandas as pd

In [367]:
file_paths = {
    "sentimento_inicial": "../original_datas/Pesquisa de Sentimento_inicial de Curso 1.xlsx",
    "feedback_andamento": "../original_datas/Pesquisa de Feedback_Andamento do Curso 1.xlsx",
    "feedback_final": "../original_datas/Pesquisa de Feedback_Avaliação Geral do Curso 1.xlsx",
}

In [368]:
sentimento_inicial = pd.read_excel(file_paths["sentimento_inicial"], sheet_name=0)
feedback_andamento = pd.read_excel(file_paths["feedback_andamento"], sheet_name=0)
feedback_final = pd.read_excel(file_paths["feedback_final"], sheet_name=0)
lista_emails = pd.read_excel(file_paths["feedback_final"], sheet_name=1)

In [369]:
def padronizar_colunas(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    return df

In [370]:
sentimento_inicial = padronizar_colunas(sentimento_inicial)
feedback_andamento = padronizar_colunas(feedback_andamento)
feedback_final = padronizar_colunas(feedback_final)
lista_emails = padronizar_colunas(lista_emails)

In [371]:
for name, df in [("Sentimento Inicial", sentimento_inicial), 
                 ("Feedback Andamento", feedback_andamento), 
                 ("Feedback Final", feedback_final)]:
    print(f"\n{name} - Hora de Início e Hora de Conclusão:")
    print(df[["hora_de_início", "hora_de_conclusão"]].info())


Sentimento Inicial - Hora de Início e Hora de Conclusão:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   hora_de_início     609 non-null    datetime64[ns]
 1   hora_de_conclusão  609 non-null    datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 9.6 KB
None

Feedback Andamento - Hora de Início e Hora de Conclusão:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   hora_de_início     354 non-null    datetime64[ns]
 1   hora_de_conclusão  354 non-null    datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 5.7 KB
None

Feedback Final - Hora de Início e Hora de Conclusão:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data c

In [372]:
sentimento_inicial.drop_duplicates(inplace=True)
feedback_andamento.drop_duplicates(inplace=True)
feedback_final.drop_duplicates(inplace=True)
lista_emails.drop_duplicates(inplace=True)

A planilha de menor entrada é a feedback_final. Logo serão utilizados apenas os 347 e-mails que constarem nas 3 planilhas.

In [373]:
lista_emails['início_da_turma'] = pd.to_datetime(lista_emails['início_da_turma'], format='%d/%m/%Y', errors='coerce')
lista_emails['final_da_turma'] = pd.to_datetime(lista_emails['final_da_turma'], format='%d/%m/%Y', errors='coerce')


In [374]:
sentimento_inicial['hora_de_início'] = pd.to_datetime(sentimento_inicial['hora_de_início'], format='%m/%d/%y %H:%M:%S', errors='coerce')
sentimento_inicial['hora_de_conclusão'] = pd.to_datetime(sentimento_inicial['hora_de_conclusão'], format='%m/%d/%y %H:%M:%S', errors='coerce')

feedback_andamento['hora_de_início'] = pd.to_datetime(feedback_andamento['hora_de_início'], format='%m/%d/%y %H:%M:%S', errors='coerce')
feedback_andamento['hora_de_conclusão'] = pd.to_datetime(feedback_andamento['hora_de_conclusão'], format='%m/%d/%y %H:%M:%S', errors='coerce')

feedback_final['hora_de_início'] = pd.to_datetime(feedback_final['hora_de_início'], format='%m/%d/%y %H:%M:%S', errors='coerce')
feedback_final['hora_de_conclusão'] = pd.to_datetime(feedback_final['hora_de_conclusão'], format='%m/%d/%y %H:%M:%S', errors='coerce')

In [375]:
print("Exemplo de início_da_turma:", lista_emails['início_da_turma'].head(3))
print("Exemplo de início_da_turma:", lista_emails['início_da_turma'].tail(3),  "\n")
print("Exemplo de hora_de_início:", feedback_final['hora_de_início'].head(3))
print("Exemplo de hora_de_início:", feedback_final['hora_de_início'].tail(3))



Exemplo de início_da_turma: 0   2024-06-02
1   2024-06-02
2   2024-06-02
Name: início_da_turma, dtype: datetime64[ns]
Exemplo de início_da_turma: 988   2024-10-14
989   2024-10-14
990   2024-10-14
Name: início_da_turma, dtype: datetime64[ns] 

Exemplo de hora_de_início: 0   2024-02-27 17:02:23
1   2024-02-27 17:04:30
2   2024-03-01 12:47:10
Name: hora_de_início, dtype: datetime64[ns]
Exemplo de hora_de_início: 344   2024-09-30 15:37:51
345   2024-09-30 17:17:11
346   2024-09-30 18:22:18
Name: hora_de_início, dtype: datetime64[ns]


In [376]:
def verificar_compatibilidade(row, lista_emails):
    # Filtrar e-mails compatíveis
    email_compat = (
        (lista_emails['e-mail_aluno'] == row['antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:']) |
        (lista_emails['e-mail_"resultados"'] == row['antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:'])
    )
    # Verificar se a hora de início está no período do curso
    periodo_compat = (
        (row['hora_de_início'] >= lista_emails['início_da_turma']) &
        (row['hora_de_início'] <= lista_emails['final_da_turma'])
    )
    # Retornar linhas compatíveis
    return lista_emails[email_compat & periodo_compat]

In [377]:
resultados_compatibilidade = []

In [378]:
for _, row in feedback_final.iterrows():
    compativeis = verificar_compatibilidade(row, lista_emails)
    if not compativeis.empty:
        for _, compat_row in compativeis.iterrows():
            merged_data = row.to_dict() | compat_row.to_dict()
            resultados_compatibilidade.append(merged_data)


In [379]:
df_mesclado = pd.DataFrame(resultados_compatibilidade)

In [380]:
print(f"Linhas compatíveis mescladas: {len(df_mesclado)}")

Linhas compatíveis mescladas: 48


Ficou claro que os e-mails, que seriam os valores únicos entre os formulários, estão incompatíveis com a lista de e-mail. Foram identificadas apenas 48 linhas do feedback final na lista de e-mails. Reconhecendo compatibilidade e-mail e período de preenchimento do formulário dentro do período de determinado curso.

Sendo assim, serão buscadas compatibilidades de e-mails entre as avaliações descartando os cursos que fizeram.
Caso um e-mail apareça mais de uma vez em uma lista de feedback, será considerado o último registro desse aluno.

In [381]:
email_counts_inicial = sentimento_inicial['antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:'].value_counts()
email_counts_andamento = feedback_andamento['antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:'].value_counts()
email_counts_final = feedback_final['antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:'].value_counts()



emails_repetidos_inicial = email_counts_inicial[email_counts_inicial > 1]
print(f"E-mails com mais de 1 ocorrência:\n{emails_repetidos_inicial.count()}")
emails_repetidos_andamento = email_counts_andamento[email_counts_andamento > 1]
print(f"E-mails com mais de 1 ocorrência:\n{emails_repetidos_andamento.count()}")
emails_repetidos_final = email_counts_final[email_counts_final > 1]
print(f"E-mails com mais de 1 ocorrência:\n{emails_repetidos_final.count()}")

E-mails com mais de 1 ocorrência:
28
E-mails com mais de 1 ocorrência:
9
E-mails com mais de 1 ocorrência:
16


In [382]:
def manter_avaliacao_mais_recentes_de_um_mesmo_usuario(df, email_col, date_col):
    df = df.sort_values(by=[email_col, date_col], ascending=[True, False])

    df_unico = df.drop_duplicates(subset=email_col, keep='first')

    return df_unico.reset_index(drop=True)

In [383]:
sentimento_inicial = manter_avaliacao_mais_recentes_de_um_mesmo_usuario(sentimento_inicial, 'antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:', 'hora_de_conclusão')
feedback_andamento = manter_avaliacao_mais_recentes_de_um_mesmo_usuario(feedback_andamento, 'antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:', 'hora_de_conclusão')
feedback_final = manter_avaliacao_mais_recentes_de_um_mesmo_usuario(feedback_final, 'antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:', 'hora_de_conclusão')

In [384]:
print(f"Linhas no Sentimento Inicial: {len(sentimento_inicial)}")
print(f"Linhas no Feedback Andamento: {len(feedback_andamento)}")
print(f"Linhas no Feedback Final: {len(feedback_final)}")

Linhas no Sentimento Inicial: 581
Linhas no Feedback Andamento: 345
Linhas no Feedback Final: 330


Por incompatibilidades dos arquivos, optei por seguir com as avaliações mais recentes para que eu possa ter um único e-mail que irá ligar as 3 avaliações.
O número caiu de 347 para 330.

In [385]:
coluna_email = 'antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:'

In [386]:
emails_final = set(feedback_final[coluna_email])
emails_andamento = set(feedback_andamento[coluna_email])
emails_inicial = set(sentimento_inicial[coluna_email])

In [387]:
emails_das_tres_avaliacoes = emails_final & emails_andamento & emails_inicial

In [388]:
sentimento_inicial_filtrado = sentimento_inicial[sentimento_inicial[coluna_email].isin(emails_das_tres_avaliacoes)]
feedback_andamento_filtrado = feedback_andamento[feedback_andamento[coluna_email].isin(emails_das_tres_avaliacoes)]
feedback_final_filtrado = feedback_final[feedback_final[coluna_email].isin(emails_das_tres_avaliacoes)]

In [389]:
emails_apenas_inicial_andamento = (emails_andamento & emails_inicial) - emails_final

In [390]:
sentimento_inicial_sem_final = sentimento_inicial[sentimento_inicial[coluna_email].isin(emails_apenas_inicial_andamento)]
feedback_andamento_sem_final = feedback_andamento[feedback_andamento[coluna_email].isin(emails_apenas_inicial_andamento)]

In [391]:
print(f"E-mails presentes em todas as 3 planilhas: {len(emails_das_tres_avaliacoes)}")
print(f"E-mails presentes em Inicial e Andamento, mas não no Final: {len(emails_apenas_inicial_andamento)}")

E-mails presentes em todas as 3 planilhas: 114
E-mails presentes em Inicial e Andamento, mas não no Final: 91


Portanto, serão mesclados os dados de 114 alunos para compor o dataset inicial que irá treinar o modelo de machine learning.

In [392]:
sentimento_inicial_filtrado.to_csv('../original_datas/sentimento_inicial_filtrado.csv', encoding="utf-8", index=False)
feedback_andamento_filtrado.to_csv('../original_datas/feedback_andamento_filtrado.csv', encoding="utf-8", index=False)
feedback_final_filtrado.to_csv('../original_datas/feedback_final_filtrado.csv', encoding="utf-8", index=False)
sentimento_inicial_sem_final.to_csv('../original_datas/sentimento_inicial_sem_final.csv', encoding="utf-8", index=False)
feedback_andamento_sem_final.to_csv('../original_datas/feedback_andamento_sem_final.csv', encoding="utf-8", index=False)

In [393]:
dataset_inicial_andamento = pd.merge(
    sentimento_inicial_filtrado,
    feedback_andamento_filtrado,
    on=coluna_email,
    suffixes=("_sentimento_inicial", "_feedback_andamento")
)

In [394]:
dataset_completo_avaliacoes = pd.merge(
    dataset_inicial_andamento,
    feedback_final_filtrado,
    left_on=coluna_email,
    right_on=coluna_email,
    suffixes=("", "_feedback_final")
)

In [395]:
print(f"Linhas no dataset unificado: {len(dataset_completo_avaliacoes)}")
print(f"Colunas no dataset unificado: {len(dataset_completo_avaliacoes.columns)}")

Linhas no dataset unificado: 114
Colunas no dataset unificado: 43


In [396]:
dataset_completo_avaliacoes.to_csv("../original_datas/dataset_completo_avaliacoes.csv", encoding="utf-8", index=False)

In [397]:
dataset_completo_avaliacoes.columns

Index(['id_sentimento_inicial', 'hora_de_início_sentimento_inicial',
       'hora_de_conclusão_sentimento_inicial', 'e-mail_sentimento_inicial',
       'nome_sentimento_inicial',
       'antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:',
       'você_acha_que_o_momento_escolhido_para_o_treinamento_é_adequado_para_você?',
       'você_pretende_participar_de_todas_as_aulas?',
       'considerando_sua_carga_de_trabalho_atual,_você_acredita_que_a_duração_das_aulas_é_apropriada?',
       'considerando_o_início_do_treinamento,_você_sente_que_tem_o_apoio_necessário_de_seus_gestores_e_colegas_para_se_concentrar_no_treinamento_durante_o_horário_das_aulas?',
       'qual_é_o_seu_nível_de_entusiasmo_em_relação_ao_conteúdo_e_objetivos_deste_treinamento?',
       'nome_da_empresa_sentimento_inicial', 'id_feedback_andamento',
       'hora_de_início_feedback_andamento',
       'hora_de_conclusão_feedback_andamento', 'e-mail_feedback_andamento',
       'nome_feedback_andamento', 'idioma

# Mascarando os dados

In [398]:
email_mapping = {email: f"meuemail{i+1}@mail.com" for i, email in enumerate(emails_das_tres_avaliacoes)}

In [399]:
dataset_completo_avaliacoes[coluna_email] = dataset_completo_avaliacoes[coluna_email].map(email_mapping)

In [400]:
coluna_cursos_de_interesse = "se_você_tivesse_a_oportunidade_de_estudar_outro_curso_da_escola,_qual_seria?"

In [401]:
def contagem_cursos_interesse(cursos):
    if pd.isna(cursos):
        return 0
    return len(cursos.split(";"))

In [402]:
dataset_completo_avaliacoes[coluna_cursos_de_interesse] = dataset_completo_avaliacoes[coluna_cursos_de_interesse].apply(contagem_cursos_interesse)

In [403]:
colunas_excluidas = [
    'id_sentimento_inicial',
    "e-mail_sentimento_inicial",
    "nome_sentimento_inicial",
    'nome_da_empresa_sentimento_inicial',
    'id_feedback_andamento',
    'e-mail_feedback_andamento',
    'nome_feedback_andamento','idioma',
    'qual_é_a_sua_percepção?1',
    'liste_abaixo_o(s)_motivo(s):',
    'nome_da_empresa_feedback_andamento',
    'id',
    'hora_de_início',
    'hora_de_conclusão',
    'cargo:',
    'como_você_avalia_a_qualidade_geral_do_curso,_incluindo_a_didática,_apresentação_do_conteúdo_e_material_fornecido?',
    'você_acha_que_a_duração_total_do_curso_foi_adequada_para_cobrir_os_tópicos_propostos?',
    'você_conseguiu_aplicar_ou_ver_aplicação_dos_conhecimentos_adquiridos_durante_o_curso_em_seu_trabalho_ou_vida_profissional?',
    'como_você_avalia_seu_próprio_aprendizado_e_desenvolvimento_ao_longo_do_curso?',
    'se_você_tivesse_a_oportunidade_de_continuar_seu_desenvolvimento_na_ferramenta_abordada_no_curso,_quão_engajado/animado(a)_você_ficaria?',
    'qual_é_a_probabilidade_de_nos_recomendar_a_um_amigo_ou_colega?',
    'deixe_aqui_seu_elogio_ou_crítica_sobre_o_curso:',
    'caso_tenha_deixado_um_depoimento_positivo,_você_nos_autoriza_a_compartilhar_em_nossas_redes_sociais?',
    'nome_da_empresa',
]

In [404]:
dataset_completo_avaliacoes.drop(colunas_excluidas, axis=1, inplace=True)

In [405]:
dataset_completo_avaliacoes.columns

Index(['hora_de_início_sentimento_inicial',
       'hora_de_conclusão_sentimento_inicial',
       'antes_de_tudo,_por_favor,_insira_seu_endereço_de_e-mail_abaixo:',
       'você_acha_que_o_momento_escolhido_para_o_treinamento_é_adequado_para_você?',
       'você_pretende_participar_de_todas_as_aulas?',
       'considerando_sua_carga_de_trabalho_atual,_você_acredita_que_a_duração_das_aulas_é_apropriada?',
       'considerando_o_início_do_treinamento,_você_sente_que_tem_o_apoio_necessário_de_seus_gestores_e_colegas_para_se_concentrar_no_treinamento_durante_o_horário_das_aulas?',
       'qual_é_o_seu_nível_de_entusiasmo_em_relação_ao_conteúdo_e_objetivos_deste_treinamento?',
       'hora_de_início_feedback_andamento',
       'hora_de_conclusão_feedback_andamento',
       'como_você_avalia_a_qualidade_da_didática_e_apresentação_do_conteúdo_até_o_momento?',
       'em_relação_ao_tópicos_que_você_acredita_que_podem_ser_melhor_explicados_ou_que_você_sente_dificuldade:',
       'você_sente_que

# Criando colunas

In [406]:
dataset_completo_avaliacoes["tempo_form_inicial_seg"] = (dataset_completo_avaliacoes["hora_de_conclusão_sentimento_inicial"] - dataset_completo_avaliacoes["hora_de_início_sentimento_inicial"]).dt.total_seconds()
dataset_completo_avaliacoes["tempo_form_andamento_seg"] = (dataset_completo_avaliacoes["hora_de_conclusão_feedback_andamento"] - dataset_completo_avaliacoes["hora_de_início_feedback_andamento"]).dt.total_seconds()

In [407]:
coluna_comentario = 'existe_algum_aspecto_do_treinamento_que_você_gostaria_de_elogiar,_destacar_ou_sugerir_melhorias?'

In [408]:
from transformers import pipeline

In [409]:
classificador = pipeline("text-classification", model="./final_model", tokenizer="./final_model")

Device set to use cpu


In [410]:
def classificador_texto(text):
    if pd.isna(text):
        return None
    result = classificador(text)[0]
    return result['label']

In [411]:
dataset_completo_avaliacoes["classificacao_comentario"] = dataset_completo_avaliacoes[coluna_comentario].apply(classificador_texto)

In [412]:
dataset_completo_avaliacoes["classificacao_comentario"]

0      LABEL_2
1      LABEL_2
2         None
3         None
4         None
        ...   
109       None
110    LABEL_0
111    LABEL_2
112    LABEL_2
113    LABEL_0
Name: classificacao_comentario, Length: 114, dtype: object

In [413]:
classificador_mapping = {
    "LABEL_0": "negativo",
    "LABEL_1": "neutro",
    "LABEL_2": "positivo"
}

In [414]:
dataset_completo_avaliacoes["classificacao_comentario"] = dataset_completo_avaliacoes["classificacao_comentario"].map(classificador_mapping)

In [415]:
dataset_completo_avaliacoes.drop([
    "hora_de_início_sentimento_inicial",
    "hora_de_conclusão_sentimento_inicial",
    "hora_de_início_feedback_andamento",
    "hora_de_conclusão_feedback_andamento",
    "existe_algum_aspecto_do_treinamento_que_você_gostaria_de_elogiar,_destacar_ou_sugerir_melhorias?"
], axis=1, inplace=True)

In [417]:
dataset_completo_avaliacoes.to_csv("./datas/dataset_completo.csv", encoding="utf-8", index=False)